# Recurrent Neural Networks
So far, the neural network architectures we have been using take in a single fixed size input and give a single fixed size output. What if we wanted to model something like language where we want to feed in different length words or sentences? Another issue is that each output is only dependent on the current input. It has no 'memory' of previous inputs so you can't model time dependent variables. Recurrent neural networks address both these issues.

They do this by having an internal hidden state which can be thought of as a form of memory. At each time step, the new hidden state is calculated as a function of the previous hidden state and the current input. This hidden state can then be used to represent your output or can be put through another function to compute the outputs. When we say function we are referring to the same one used in standard neural network: linear combination followed by an activation function.

### $h_t = f(x_t, h_{t-1})$

As shown in the diagram below, which uses a further function to compute the output $o$ from the hidden state $s$, there are three matrices of parameters which we are trying to optimize: U, V and W. The diagram also demonstrates how these networks can be unfolded to show the variables at various time steps.

![image](images/rnn.jpg)

Standard neural networks can only model one to one relationships while RNNs are extremely flexible in terms of input-output structures which is one of the reasons they are so powerful. You can imagine something like one to many being used to feed in a single image from which a caption is sequentially produced or a many to one being used to feed in a sentence sequentially and give a single output describing the sentiment of the sentence.

![image](images/rnnlayouts.jpeg)

### Optimization
Surprisingly, with this increased complexity in structure, the optimization method does not become any more difficult. Despite having a different name, back-propagation through time, it is essentially the same thing. All you do is feed in your sequence sequentially to get the output, as usual. You then just calculate your error at each timestep and sum it as opposed to calculating the error at a single timestep like standard neural networks. Then you can use gradient descent to update your weights iteratively until you are satisfied with your network's performance.

RNNS are generally slower to optimize than standard neural networks as the output at each time step is dependent on the previous output so the operations cannot be parallelized.

For a long time it was considered difficult to train RNNs due to two problems called vanishing and exploding gradients. These problems also exist in standard neural network but are greatly emphasized in RNNs. However, modern techniques such as LSTM cells have greatly reduced this difficulty.

For this particular task, we will need to do quite a bit of pre-processing. We need to find the number of unique characters in our training text and give each one a unique number so we can one-hot encode them.<br>
We start by reading the file, converting all letters to lowercase to reduce the number of characters we need to model, then defining a function which takes in the text and gives up back a dictionary mapping each letter to a unique number.

## Implementation
We are going to be implementing a one-to-one character level text prediction model. We will be sequentially feeding in a single character and asking our network to predict the next character as a time dependent function of all the characters that came before it.

As always, we begin by importing the required libraries.

In [1]:
import torch
import numpy as np
import torch.nn.functional as F

Lets read in our dataset. We just need a text file which contains the data which we want to model. In this case, I have found a file which contains ~0.5MB of Kendrick Lamar lyrics. You can use a variety of different datasets. There are plenty of which are easily accessible online - check out the links below. Otherwise, is very easy to create your own either by copying and pasting text into a file or creating a bot to automatically do this for you.

[Datasets repo 1](https://github.com/cedricdeboom/character-level-rnn-datasets/tree/master/datasets)

In [2]:
#open our text file and read all the data into the rawtxt variable
with open('lyrics.txt', 'r') as file:
    rawtxt = file.read()

#turn all of the text into lowercase as it will reduce the number of characters that our algorithm needs to learn
rawtxt = rawtxt.lower()

#returns a dictionary that allows us to map from a unique number to a unique character in our text
def create_map(rawtxt):
    letters = set(rawtxt) #returns the list of unique characters in our raw text
    lettermap = dict(enumerate(letters)) #created the dictionary mapping
    return lettermap

num_to_let = create_map(rawtxt) #store the dictionary mapping from numbers to characters in a variable
let_to_num = dict(zip(num_to_let.values(), num_to_let.keys())) #create the reverse mapping so we can map from a character to a unique number

nchars = len(num_to_let) #number of unique characters in our text file
print('Size of dataset:', len(rawtxt))
print('Number of unique characters:', nchars)

Size of dataset: 24442
Number of unique characters: 56


We now define a function which takes in text and a dictionary and maps each character in the text to the value specified for it in the dictionary. We then use this to map all of our text into the unique numbers for each character so it can be used with our RNN model. The labels are specified as the input but shifted by one time step as the label for each input is the character which comes after it.

We also define the function which returns a random chunk from the text which we can use to train out model.

In [3]:
def maparray(txt, mapdict):
    
    txt = list(txt)

    #iterate through our text and change the value for each character to its mapped value
    for k, letter in enumerate(txt):
        txt[k] = mapdict[letter]

    txt = np.array(txt)
    return txt

#map our raw text into our input variables using the function defined earlier and passing in the mapping from letters to numbers
X = maparray(rawtxt, let_to_num)
Y = np.roll(X, -1, axis=0) #our label is the next character so roll shifts our array by one timestep

#conver to torch tensors and make row vectors so we can use them in our torch model
X = torch.LongTensor(X).unsqueeze(dim=1)
Y = torch.LongTensor(Y).unsqueeze(dim=1)

In [4]:
#return a random batch for training
def random_chunk(chunk_size):
    k = np.random.randint(0, len(X)-chunk_size)
    return X[k:k+chunk_size], Y[k:k+chunk_size]

In [5]:
x, y = random_chunk(10)
x.shape, y.shape

(torch.Size([10, 1]), torch.Size([10, 1]))

Define our model which takes in variables defining its structure as parameters. The encoder converts each unique number into an embedding which is fed into the rnn model. The RNN calculates the hidden state which is converted into an output through a fully connected layer called the decoder.<br>
We also define the init_hidden function which outputs us a tensor of zeros of the required size for the hidden state.

In [58]:
class ChaRNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.encoder = torch.nn.Embedding(input_size, input_size)
        self.i2h = torch.nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        x = self.encoder(x.view(-1)) #encode our input into a vector embedding
        combined = torch.cat((x, hidden), 1)
        hidden = torch.tanh(self.i2h(combined))
        output = self.i2o(hidden)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)
    
class ChaRNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super().__init__()
        #store input parameters in the object so we can use them later on
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        #required functions for model
        self.encoder = torch.nn.Embedding(input_size, input_size)
        self.rnn = torch.nn.RNN(input_size, hidden_size, n_layers, batch_first=True)
        self.decoder = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden_state):
        x = self.encoder(x.view(-1, 1)) #encode our input into a vector embedding
        x, hidden_state = self.rnn(x.view(1, 1, self.input_size), hidden_state) #calculate the output from our rnn based on our input and previous hidden state
        x = self.decoder(x.view(1, -1)) #calculate our output based on output of rnn
        return x, hidden_state

    def init_hidden(self):
        return torch.zeros(self.n_layers, 1, self.hidden_size) #initialize our hidden state to a matrix of 0s

Instantiate our model, define the appropriate hyper-parameters, cost function and optimizer. We will be training on ranom samples from the text of length chunk_size so it is what batch size is to normal neural networks.

In [59]:
#hyper-params
lr = 0.001
epochs = 50
chunk_size = 100 #the length of the sequences which we will optimize over

myrnn = ChaRNN(nchars, 512, nchars) #instantiate our model from the class defined earlier
criterion = torch.nn.CrossEntropyLoss() #define our cost function
optimizer = torch.optim.Adam(myrnn.parameters(), lr=lr) #choose optimizer

# SET UP TRAINING VISUALISATION
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter() # we will use this to show our models performance on a graph

Define the axes for plotting our cost per epoch. Define the training loop, sequentially feeding in a random chunk of text, summing the cost for each character in the sequence (backpropagation through time) and calculating the gradients to update our weights.

In [60]:
#training loop
def train(epochs):
    for epoch in range(epochs):
        total_loss = 0 #stores the cost per epoch
        generated = '' #stores the text generated by our model each epoch
        #given our chunk size, how many chunks do we need to optimize over to have gone thorough our whole dataset
        for idx in range(len(X)//chunk_size):
            h = myrnn.init_hidden() #initialize our hidden state to 0s
            loss = 0 #cost for this chunk
            x, y = random_chunk(chunk_size) #get a random sequence chunk to train
            #sequentially input each character in our sequence and calculate loss
            for i in range(chunk_size):
                out, h = myrnn.forward(x[i], h) #calculate outputs based on input and previous hidden state

                #based on our output, what character does our network predict is next?
                _, outl = out.data.max(1)
                letter = num_to_let[outl.item()]
                generated+=letter #add the predicted letter to our generated sequence
                loss += criterion(out, y[i]) #add the cost for this input to the cost for this current chunk
            
            writer.add_scalar('Loss/Train', loss/chunk_size, epoch*(len(X)//chunk_size) + idx)    # write loss to a graph
            
            #based on the sum of the cost for this sequence (backpropagation through time) calculate the gradients and update our weights
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss+=loss.item() #add the cost of this sequence to the cost of this epoch
        total_loss /= len(X) #divide by the number of chunks per epoch to get average cost per epoch

        print('Epoch ', epoch+1, ' Avg Loss: ', total_loss)
        print('Generated text: ', generated[0:750], '\n')
        
train(epochs)

Epoch  1  Avg Loss:  2.4635538214035915
Generated text:  ’5/7f7p.ydx]dy]x0y')x7y[dumf’:d9x”di]uo’70d’5/:9ubd 1ś7/y’cyd)." yx”du7]wuy/y’o)p 9’m0)/:“70j/:q5xd""bf/ don fa dopdoo)z)0 cdondo,"dod nddoeb)do 0do )donpdo oydo wyd cznzydo]do yxedon fonxdonqn onx dofion  do ndon d os do kdo  ng ands odo oy s do ndon dodoongdondo ndo n on d do  do nc  ofyan  dn adodo  o d t  don  do    o   neo on on   don    do n     do don  n  don    do o ndos odon     do   do   c  o   o     o    o   o         o  o    o     o     o  o    o          do   o    o      on    o    do   o            o       o  o  o  o       o  o    o   o        o     o         o  o    o           aou     o          o     o    o   o            o   o    o    o   o    o   o   o  o              o   do oo o   o oo o   o oo o   o oo o  oo  o   o  o   

Epoch  2  Avg Loss:  2.054762767463686
Generated text:  ghdhdow  wol' up,litebitch)
(e wotele
thotch)
(hol' up, hol' up, hol' up, hol' up,
hotch)ltthdown (htgot aoale   i wnwust wonl teot ho

Epoch  12  Avg Loss:  0.8862822770818771
Generated text:  nk iaitica oant th santion chet the  fane aher iis hop was wapping,monherfucker if you wid, then bil w iou  iarte in  aybit le bit ooma hogh tykntid n e, ih
averybody waoedbabtc, mou cike iife
in sndmeaown fi thes iau śe with yy mao bor
 thpter wike iiot ain tor the ciamd   
eitla i can gut aour en n  tn  by i sotd, ine
iog homie sa to  tintitive, i 'apill out th the careet,
i gake ihe gord abd bruihou  srckeng (oar this chat trt th
  idd ,h
te 
then that ttgot, i got, i got, i got,loaloes , i giow just, know just, know just, know just, know just,what you want poetic justice, pot it in a song
s 'loght as the fortin' ohn
gioon tkinged  mlt you  ploe syes toll me iour eama tan t aun sieak (disther you  paby iiller mi a couard
i can t tver kne 

Epoch  13  Avg Loss:  0.8006105780767327
Generated text:  o huneng  ine
i dtey totin,,iiot tt, iye
iho wlatiryte it, i e
ihis ghet ieey diwcin
 phet pvenns thkolwyne
 bit i rin't shrees ng


Epoch  23  Avg Loss:  0.44457402669040263
Generated text:   devter iere s true
livin' iy lile in thi horgin and that we  phomewas cribf
i'm talking 'oetic just  tt,aou can get it
 you can get it
and i know just, know just, know just, know just, know just,what le iou snd iour namma ta the serherfand  i could bo it
maybe one say when you wugure out iou re iont k she cames  ievel numbers9
took wp tn the hty, 10 it on the sas
yenten e in the das
 killin's in tocdteown skinned, bet your hlue eyes tell me iour myma can't sun sneak adissin')
sneak me through tkitnd pared of the photoshop
show me some hing matural like asro on aichard pryor
mhow me some hing  e set si don't conpromise, itjust wenetrate
sex, money, murder these sre the bieaks
the e are the b deoteere
you should chip a nigga, then throw the  

Epoch  24  Avg Loss:  0.4210524312428177
Generated text:  n (eere yy a countant tives
an tact i'm donn at this
dausśe with my poo bae, tastes like kool aid foinbon i cane to there you wesi

Epoch  34  Avg Loss:  0.3587986712428047
Generated text:   coe i once you let me uo the extra 
pull ip on your block, then breaksit down we playing tetris
a.m il’coa tay too cetty once you let me uo the extra 
pull up on your block, then breaksit down we placonja
 gimme some ganja
real nigga in my dna
ain't no ho inside my dna 

k dot, pack up the phone, nunteuou
this is pauna's ofdest son
i know murser  condietion
burner , boos ers, boty ars, ballers, d ,twing kunta) everybody wanna cut the legs off him (king kunta) kunta
black man taking no losses
biwverbhy  there's blood,in my pen
better yes, where your driends and beme inreally wanna know you dnli you lon' take it)
we want the funk?(no you want the funk?)
we want the funk?(no you want the funk?nh,ihat you want
poetic justice, put it in a song
 

Epoch  35  Avg Loss:  0.36457489559297507
Generated text:   tonl lid iettthe camst onvorved tuen history repeats
but i resolved inside that private hall while oi  npot and watch him hit his

KeyboardInterrupt: 

The generated text above picks the most probable next character each time. This is not the best way to do it as our model will be deterministic so it will produce the same text over and over again. To get it producing different text, we should instead sample from the probability distribution of possible next letters output by the network. That is what we will do with the next function.

In [61]:
def generate(prime_str='a', str_len=150, temperature=0.75):
    generated = prime_str
    
    #initialize hidden state
    h = myrnn.init_hidden()
    
    prime_str = maparray(prime_str, let_to_num)
    x = torch.LongTensor(prime_str)
    
    #primes our hidden state with the input string
    for i in range(len(x)):
        out, h = myrnn.forward(x[i].unsqueeze(0), h)
    
    x = x[-1]
    
    for i in range(str_len):
        out, h = myrnn.forward(x, h)
        
        out_dist = out.data.view(-1).div(temperature).exp()
        sample = torch.multinomial(out_dist, 1).item()
        pred_char = num_to_let[sample]
        
        generated += pred_char
        
        x = torch.LongTensor([sample])
    
    return generated
        
    

gen = generate('this be ', 2000, 0.75)
print(gen)

this be uck about my fors
with lover through the cameras
eath the uck around my some compton"
i should probably go to schorlin' as inside my dna
i just win again, then win again, then you crippin' get it on my camerican flag

i don't conversat
yo shitshunl up you can get it, you can get it
and i know we inside my dna
cocaine quarter piece, got war and place, flow, 'ip's i silled do secs
it bacal time i call, it's a nigga little bitty range i how hup the projever of dedicmainside your dna plead the same
rater was proof
i'm talking poetic justice, poetic justice
if i told you that mean somebody gettin' killin' a thine kines

i readdac is erembels off him (king kunta) kunta
black man taking no losses
bitch where you reside
and looked around to school with, boo boo
your home boy, your block that you're from chel' of sharks, he lledd be within colibu of for sool no mo', no mo' (no mo')
what's go nothin'
i'll chip a nigga little bitty range i hol' up lil bitch) be humble
(hol' up bitch) sit 